In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('kme').getOrCreate()

In [3]:
df = spark.sql('select * from seeds_dataset')

In [4]:
df.show(5)

+-----+---------+-----------+------------------+------------------+---------------------+----------------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient|length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+----------------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|
14.88| 14.57| 0.8811| 5.553999999999999| 3.333| 1.018| 4.956|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|
13.84| 13.94| 0.8955| 5.324|3.3789999999999996| 2.259| 4.805|
16.14| 14.99| 0.9034|5.6579999999999995| 3.562| 1.355| 5.175|
+-----+---------+-----------+------------------+------------------+---------------------+----------------+
only showing top 5 rows

In [5]:
from pyspark.ml.clustering import KMeans

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols=['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove'],outputCol='features')

In [8]:
output = assembler.transform(df)

In [9]:
df2 = output.select('features')

In [10]:
df2.show(5)

+--------------------+
 features|
+--------------------+
[15.26,14.84,0.87...|
[14.88,14.57,0.88...|
[14.29,14.09,0.90...|
[13.84,13.94,0.89...|
[16.14,14.99,0.90...|
+--------------------+
only showing top 5 rows

In [11]:
#scalling the data
from pyspark.ml.feature import StandardScaler

In [12]:
scaler = StandardScaler(inputCol='features',outputCol='scaled_features')
final_data = scaler.fit(df2).transform(df2)

In [13]:
#creating the model
kmeans = KMeans(featuresCol='scaled_features',k=3,seed=1)

In [14]:
model = model.fit(final_data)

In [15]:
model.clusterCenters()

Out[ 31 ]: 
[array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,
 9.7892399 , 2.41585013, 12.29286107]),
 array([ 4.07135818, 10.14438097, 35.86461803, 11.81349589,
 7.53471695, 3.18317127, 10.39230304]),
 array([ 4.94114963, 10.95557919, 37.3028184 , 12.42383591,
 8.60815545, 1.80983376, 10.40657797])]

In [16]:
# within set sum of squared error
wssse = model.computeCost(final_data)
print(wssse)

428.633343229

In [17]:
# lets assign a lable with each datapoint
result = model.transform(final_data)
result.show(10)

+--------------------+--------------------+----------+
 features| scaled_features|prediction|
+--------------------+--------------------+----------+
[15.26,14.84,0.87...|[5.24452795332028...| 2|
[14.88,14.57,0.88...|[5.11393027165175...| 2|
[14.29,14.09,0.90...|[4.91116018695588...| 2|
[13.84,13.94,0.89...|[4.75650503761158...| 2|
[16.14,14.99,0.90...|[5.54696468981581...| 2|
[14.38,14.21,0.89...|[4.94209121682475...| 2|
[14.69,14.49,0.87...|[5.04863143081749...| 2|
[14.11,14.1,0.891...|[4.84929812721816...| 2|
[16.63,15.46,0.87...|[5.71536696354628...| 0|
[16.44,15.25,0.88...|[5.65006812271202...| 2|
+--------------------+--------------------+----------+
only showing top 10 rows